## The second In-class-exercise (09/22/2022, 40 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points): Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [11]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
Area of Problem:
My area of problem is to explore the text data that secure from the evaluation of biographhy of population

Research question:
. Do the concept Modelling of the concepts that relaters discussed about throughout the evaluation.
. Do  Year of Birth, EDA on the Location, Generation data of the relaters, Gender.

Data Required for answer the questions:
 Below are the attributes of the necessary data to answer the research question
. Bio of the relater
. Transcript characteristics such as evaluation questions and answers 

We can collect 1042 evaluation samples from the below link: https://ddr.densho.org/narrators/

Steps for collecting and saving the data:
. We use web scraping concept to push the data.
. We use BeautifulSoup library to analyze the Densho Digital Repository.
. There are 37 pages of relaters.
. We analyze between every one page and go to the relater subject link and then to evaluations hyperlink.
. We get the bio from the relater homepage.
. The evaluation portion characteristics are obtained from the transcript hyperlink.
. We analyze the Body serction-> Passage components in the html code and then store the text data in a json structure.
. The json structure is put together the data for our research.

'''

'\nPlease write you answer here:\nArea of Problem:\nMy area of problem is to explore the text data that secure from the evaluation of biographhy of population\n\nResearch question:\n. Do the concept Modelling of the concepts that relaters discussed about throughout the evaluation.\n. Do  Year of Birth, EDA on the Location, Generation data of the relaters, Gender.\n\nData Required for answer the questions:\n Below are the attributes of the necessary data to answer the research question\n. Bio of the relater\n. Transcript characteristics such as evaluation questions and answers \n\nWe can collect 1042 evaluation samples from the below link: https://ddr.densho.org/narrators/\n\nSteps for collecting and saving the data:\n. We use web scraping concept to push the data.\n. We use BeautifulSoup library to analyze the Densho Digital Repository.\n. There are 37 pages of relaters.\n. We analyze between every one page and go to the relater subject link and then to evaluations hyperlink.\n. We get

Question 2 (10 points): Write python code to collect 1000 data samples you discussed above.

In [18]:
# You code here (Please add comments in the code):
# The below is for collectig the 1042 samples from 904 narrators from the https://ddr.densho.org/narrators
# We collect the data and save it in JSON format.

# importing the libraries...
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import json
import re

total_count = 0
count = 0
interview_count = 0
# url of the densho digital repository to scrape the data
main_url = "https://ddr.densho.org/narrators/?page={}"
for page_num in range(1, 38):
    # we create the data soup of the main url page html elements.
    link1 = Request(main_url.format(page_num), headers={'User-Agent': 'Mozilla/5.0'})
    url1 = urlopen(link1)

    data1 = url1.read()
    data1_soup = BeautifulSoup(data1)
    
    #print("*** Page {} ***".format(page_num))
    # we iterate through each narrator page link
    for narrator_link in data1_soup.find_all('h4'):
        link2 = Request(narrator_link.a.get('href'), headers={'User-Agent': 'Mozilla/5.0'})
        url2 = urlopen(link2)

        data2 = url2.read()
        data2_soup = BeautifulSoup(data2)
        count = count + 1
        
        # *** Interview links page***
        # we find the div and attributes of the div and fetch the required values
        narrator = data2_soup.find_all("div", attrs={'class':'col-sm-8 col-md-8'})[0]
        if data2_soup.find_all("div", attrs={'class':'url'}) != []:
            
            # Some narrators have given than one interview
            # we iterate through each interview of the narrator
            for interviews in data2_soup.find_all("div", attrs={'class':'url'}):
                result = {}
                result['Narrator_Name'] = ""
                result['Bio'] = ""
                result['Title'] = ""
                result['Interviewer'] = ""
                result['Location'] = ""
                result['Date'] = ""
                result['Densho ID'] = ""
                result['transcript'] = {}
                # Narrator name and Bio...
                try:
                    result['Narrator_Name'] = narrator.h1.text.strip().replace('"', "")#re.sub('"', '\"', narrator.h1.text.strip())
                except:
                    result['Narrator_Name'] = ""
                    
                try:
                    result['Bio'] = narrator.p.text
                except :
                    result['Bio'] = ""
                
                # we go to interview page and fetch details from it
                link3 = Request(interviews.a.get('href'), headers={'User-Agent': 'Mozilla/5.0'})
                url3 = urlopen(link3)

                data3 = url3.read()
                data3_soup = BeautifulSoup(data3)
                #print("*** Segment Descriptions ***")
                num = 1
                # we go to the 'tbody' element and find all 'td' elements
                for desc in data3_soup.find_all('tbody')[0].find_all('td'):
                    result['transcript']['segment {}'.format(num)] = {}
                    result['transcript']['segment {}'.format(num)]['Segment Desc'] = desc.text.split(' (', 1)[0].strip()
                    result['transcript']['segment {}'.format(num)]['Segment Q&A'] = {}
                    num = num + 1
                
                for transcript_link in data3_soup.find_all('div', attrs={'id':'dls_tab'}):
                    #print("*** Transcript link ***")
                    try:
                        #print(transcript_link.find_all('a')[1].get('href'))
                        link4 = Request(transcript_link.find_all('a')[1].get('href'), headers={'User-Agent': 'Mozilla/5.0'})
                        url4 = urlopen(link4)

                        data4 = url4.read()
                        data4_soup = BeautifulSoup(data4)

                        #print("-----------------------------------------")
                        for data in data4_soup.find_all('div', attrs={'class':'segmentHead'}):
                            # title, interviewer, location, date, densho id...
                            for attr in data.text.splitlines():
                                if 'Title' in attr:
                                    #print(i.strip())
                                    result['Title'] = ' '.join(attr.strip().split()[1:])
                                if 'Interviewer' in attr:
                                    #print(i.strip())
                                    result['Interviewer'] = ' '.join(attr.strip().split()[1:])
                                if 'Location' in attr:
                                    #print(i.strip())
                                    result['Location'] = ' '.join(attr.strip().split()[1:])
                                if 'Date' in attr:
                                    #print(i.strip())
                                    result['Date'] = ' '.join(attr.strip().split()[1:])
                                if 'Densho ID' in attr:
                                    #print(i.strip())
                                    result['Densho ID'] = ' '.join(attr.strip().split()[2:])
                        
                        num = 1
                        # An interview has multiple segments
                        # Each segment has questions and answer
                        for seg in data4_soup.find_all('div', attrs={'class':'segmentBody'}):
                            # Each segment will have multiple paragraph elements
                            for p in range(0, len(seg.find_all('p')[1:-1])):
                                if '?' in seg.find_all('p')[p].text:
                                    question = re.sub('\s+',' ', re.sub("  +", "",seg.find_all('p')[p].text))
                                    question = " ".join(i for i in question.split()[1:])
                                    try:
                                        # we remove unnecessary white spaces...
                                        answer = re.sub('\s+',' ', re.sub("  +", "",seg.find_all('p')[p+1].text))
                                        answer = " ".join(i for i in answer.split()[1:])
                                    except:
                                        answer = ""
                                    result['transcript']['segment {}'.format(num)]['Segment Q&A'][question] = answer
                            num = num + 1
                                                
                    #except HTTPError as err:
                    except Exception as e:
                        #print("No Transcripts")
                        #print(e)
                        #print(len(data3_soup.find_all('tbody')[0].find_all('td')))
                        result['Title'] = ""
                        result['Interviewer'] = ""
                        result['Location'] = ""
                        result['Date'] = ""
                        result['Densho ID'] = ""
                        #num = 1
                        result['transcript'] = {}
                        for num in range(0, len(data3_soup.find_all('tbody')[0].find_all('td'))):
                            result['transcript']['segment {}'.format(num+1)] = {}
                            result['transcript']['segment {}'.format(num+1)]['Segment Desc'] = ""
                            result['transcript']['segment {}'.format(num+1)]['Segment Q&A'] = {}
                            #num = num + 1
                '''
                import os.path
                if os.path.isfile(r'{}.json'.format(result['Narrator_Name'] + '-' + str(interview_count))):
                    with open(r'{}.json'.format(result['Narrator_Name'] + '(1)' + '-' + str(interview_count)), 'w', encoding='utf-8') as f:
                        json.dump(result, f, ensure_ascii=False, indent=4)
                else:
                    with open(r'{}.json'.format(result['Narrator_Name'] + '-' + str(interview_count)), 'w', encoding='utf-8') as f:
                        json.dump(result, f, ensure_ascii=False, indent=4)
                print('***************************')
                '''
                interview_count = interview_count + 1 
             
        else:
            # some narrators have no interviews
            # In that case, we fill the json file with empty values.
            '''
            print("%%%%%%%%%%%%%%%%%%%%%%%%%")
            print("No Interviews....")
            print("%%%%%%%%%%%%%%%%%%%%%%%%%")
            '''
            result= {}
            result['Narrator_Name'] = narrator.h1.text.strip().replace('"', '')#re.sub('"', '\"', narrator.h1.text.strip())
            result['Bio'] = narrator.p.text
            result['Title'] = ""
            result['Interviewer'] = ""
            result['Location'] = ""
            result['Date'] = ""
            result['Densho ID'] = ""
            result['transcript'] = {}
            interview_count = interview_count + 1
            #print(result)
            '''
            with open(r'{}.json'.format(result['Narrator_Name']), 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=4)
                        
            print("****************************")    
            '''
    
print("Total count of narrators is: {}".format(count))
print("Total count of Samples is: {}".format(interview_count))

Total count of narrators is: 918
Total count of Samples is: 1064


Question 3 (10 points): Write python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "information retrieval". The articles should be published in the last 10 years (2012-2022).

The following information of the article needs to be collected:

(1) Title

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [5]:
# You code here (Please add comments in the code):
import json as j
import urllib.request as request
from bs4 import BeautifulSoup as beauty

with request.urlopen('https://api.semanticscholar.org/v1/paper/10.1145/2348283.2348407') as fileread:
    page = fileread.read()
    v = beauty(page)
    print(v.p.text)
    with open('output.txt', 'w') as filewrite:
        j.dump(s.p.text, filewrite)



{"abstract":"Typically, every part in most coherent text has some plausible reason for its presence, some function that it performs to the overall semantics of the text. Rhetorical relations, e.g. contrast, cause, explanation, describe how the parts of a text are linked to each other. Knowledge about this so-called discourse structure has been applied successfully to several natural language processing tasks. This work studies the use of rhetorical relations for Information Retrieval (IR): Is there a correlation between certain rhetorical relations and retrieval performance? Can knowledge about a document's rhetorical relations be useful to IR? We present a language model modification that considers rhetorical relations when estimating the relevance of a document to a query. Empirical evaluation of different versions of our model on TREC settings shows that certain rhetorical relations can benefit retrieval effectiveness notably (>10% in mean average precision over a state-of-the-art b

Question 4 (10 points): Write python code to collect 1000 posts from Twitter, or Facebook, or Instagram. You can either use hashtags, keywords, user_name, user_id, or other information to collect the data. 

The following information needs to be collected:

(1) User_name

(2) Posted time

(3) Text 

In [12]:
# You code here (Please add comments in the code):

import os
import tweepy 
import pandas as pd

x= '2Em7SxlX9jPMfL4x97r3zMO0x'
y= 'sVbJzekKuiAgq83Y7gCwNVbSowqQokGVzWexKHl2cXIPceWtSd'
z= '1439767876962029572-uUMt8oWRyzj9ilE5zk4uYbL93sCMPT'
perm= 'oydIGymn9bS767FVEMawE9GyGAnMmBJfaY2XXKmHnmliF'

authorize = tweepy.OAuthHandler(x, y)
authorize.set_access_token(z, perm)
i = tweepy.API(authorize, wait_on_rate_limit=True)

word = "#love"
date = "2021-12-10"

twee = tweepy.Cursor(i.search, q=word, lang="en", since=date).items(1000)

[i.text for i in twee]


['T-shirt for Couple Who love Each other ❤ Get From Here https://t.co/4SOxu50PuQ  \n\n#couple #love #lovetshirt… https://t.co/9j665KWnMM',
 'RT @assessorito: YOU ARE BEAUTIFUL SMILE BE HAPPY :P WOW #LOVE https://t.co/7pvwS4TZHu',
 'RT @Soulcandi78: The personal is political!  Black dignity &amp; black pride critical to the realisation of the mission to liberate AZANIA. #so…',
 'RT @Kari8120: Support healthy weight loss with a variety of proven remedies.\nGo here: https://t.co/TAnfhTudTW\n\n#weight #loss #fitness #love…',
 'RT @KariJoys: Always stay #Humble &amp; #Kind \n\n#JoyTrain #Joy #Love #Kindness #Quote #Blessed #MentalHealth #Mindfulness #GoldenHearts #IAM #S…',
 'RT @InnerPeaceThpy: LOVE Energy - Heart Chakra 💗\n\nhttps://t.co/FznkvJzBgy \n\n#innerpeacesoundtherapy #innerpeacesoundthpy #innerpeacethpy #l…',
 'LovFash_Grl: JolieNash8: Follow: https://t.co/4W2uil5VZ1  (for way more cuteness!!) \n\n#cute #cat #dog #animal #pet… https://t.co/EWaITrp8Xu',
 'Happiness is a direction,